In [1]:
from typing import TypedDict, List
from langgraph.graph import START, END, StateGraph

In [2]:
import operator
from tracemalloc import start
from typing import final


class AgentState(TypedDict):
    number1: int
    operation: str
    number2: int
    number3: int
    operation2: str
    number4: int
    finalNumber: int
    finalNumber2: int

In [3]:
def adder(state: AgentState) -> AgentState:
    """This node adds the 2 numbers"""
    
    state['finalNumber'] = state['number1'] + state['number2']

    return state

def subtractor(state: AgentState) -> AgentState:
    """This node substracts the 2 numbers"""
    state['finalNumber'] = state['number1'] - state['number2']
    return state

In [4]:
def adder2(state: AgentState) -> AgentState:
    """This node adds the 2 numbers"""
    
    state['finalNumber2'] = state['number3'] + state['number4']

    return state

def subtractor2(state: AgentState) -> AgentState:
    """This node substracts the 2 numbers"""
    state['finalNumber2'] = state['number3'] - state['number4']
    return state

In [5]:
def route1(state: AgentState) -> AgentState:
    if state['operation'] == '+':
        return 'addition_operation'
    elif state['operation'] == '-':
        return 'subtraction_operation'

def route2(state: AgentState) -> AgentState:
    if state['operation2'] == '+':
        return 'addition_operation2'
    elif state['operation2'] == '-':
        return 'subtraction_operation2'

In [6]:
graph = StateGraph(AgentState)
graph.add_node('add1', adder)
graph.add_node('subtract1', subtractor)
graph.add_node('add2', adder2)
graph.add_node('subtract2', subtractor2)
graph.add_node('router1', lambda state:state)
graph.add_node('router2', lambda state:state)

graph.add_edge(START, 'router1')
graph.add_conditional_edges(
    'router1',
    route1,
    {
        'addition_operation': 'add1',
        'subtraction_operation': 'subtract1'
    }
) # remeber to use 'add_conditional_edge' to set up a conditional_node
graph.add_edge('subtract1', 'router2')
graph.add_edge('add1', 'router2')
graph.add_conditional_edges(
    'router2',
    route2,
    {
        'addition_operation2': 'add2',
        'subtraction_operation2': 'subtract2'
    }
) # remeber to use 'add_conditional_edge' to set up a conditional_node
graph.add_edge('subtract2', END)
graph.add_edge('add2', END)


app = graph.compile()

In [7]:
new_state = AgentState({'number1': 10, 'operation': "-", 'number2': 5, 'number3': 7, 'number4': 2, 'operation2': '+', 'finalNumber': 0, 'finalNumber2': 0})
print(app.invoke(new_state))

{'number1': 10, 'operation': '-', 'number2': 5, 'number3': 7, 'operation2': '+', 'number4': 2, 'finalNumber': 5, 'finalNumber2': 9}
